1. Loading the libraries

In [10]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [62]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [12]:
emotion_df=pd.read_csv("emotion.csv")

In [13]:
emotion_df.head()

,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4


In [14]:
violence_df=pd.read_csv("violence.csv")

In [15]:
violence_df.head()

,Tweet_ID,tweet,type
0,ID_0022DWKP,Had a dream i got raped last night. By a guy i...,sexual_violence
1,ID_00395QYM,he thought the word raped means sex and told m...,sexual_violence
2,ID_003EOSSF,She NOT TALKING TO ME I WAS RAPED BY 2 MEN 1 M...,sexual_violence
3,ID_004BBHOD,I was sexually abused for 3 years at age 4 to ...,sexual_violence
4,ID_004F7516,Chessy Prout can do better by telling the trut...,sexual_violence


In [16]:
hate_df=pd.read_csv("hatespeech.csv")

In [17]:
hate_df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


2. Data Preprocessing

In [18]:
emotion_df.drop(columns=['Unnamed: 0'],inplace=True)

In [19]:
emotion_df.head()

,text,label
0,i just feel really helpless and heavy hearted,4
1,ive enjoyed being able to slouch about relax a...,0
2,i gave up my internship with the dmrg and am f...,4
3,i dont know i feel so lost,0
4,i am a kindergarten teacher and i am thoroughl...,4


In [20]:
violence_df.drop(columns=['Tweet_ID'],inplace=True)

In [21]:
violence_df.head()

,tweet,type
0,Had a dream i got raped last night. By a guy i...,sexual_violence
1,he thought the word raped means sex and told m...,sexual_violence
2,She NOT TALKING TO ME I WAS RAPED BY 2 MEN 1 M...,sexual_violence
3,I was sexually abused for 3 years at age 4 to ...,sexual_violence
4,Chessy Prout can do better by telling the trut...,sexual_violence


In [22]:
hate_df.drop(columns=['Unnamed: 0'],inplace=True)

In [23]:
hate_df=hate_df[['tweet','class']]

In [24]:
hate_df.head()

,tweet,class
0,!!! RT @mayasolovely: As a woman you shouldn't...,2
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1


In [25]:
emotion_df.columns,violence_df.columns,hate_df.columns

(Index(['text', 'label'], dtype='object'),
 Index(['tweet', 'type'], dtype='object'),
 Index(['tweet', 'class'], dtype='object'))

In [26]:
violence_df.rename(columns={'tweet' : 'text','type':'label'},inplace=True)
hate_df.rename(columns={'tweet' : 'text','class':'label'},inplace=True)

In [27]:
emotion_df.columns,violence_df.columns,hate_df.columns

(Index(['text', 'label'], dtype='object'),
 Index(['text', 'label'], dtype='object'),
 Index(['text', 'label'], dtype='object'))

In [28]:
emotion_df.isna().sum(),violence_df.isna().sum(),hate_df.isna().sum()

(text     0
 label    0
 dtype: int64,
 text     0
 label    0
 dtype: int64,
 text     0
 label    0
 dtype: int64)

In [29]:
emotion_df.shape,violence_df.shape,hate_df.shape

((416809, 2), (39650, 2), (24783, 2))

In [30]:
emotion_df['label'].value_counts()

label
1    141067
0    121187
3     57317
4     47712
2     34554
5     14972
Name: count, dtype: int64

In [31]:
e_df=pd.DataFrame()
for i in range(6) :
    subset=emotion_df[emotion_df['label']==i].sample(n=2000,random_state=42)
    e_df=pd.concat([e_df,subset])

In [32]:
e_df.shape

(12000, 2)

In [33]:
emotion_df=e_df.copy()

In [34]:
emotion_df['label'].value_counts()

label
0    2000
1    2000
2    2000
3    2000
4    2000
5    2000
Name: count, dtype: int64

In [35]:
violence_df['label'].value_counts()

label
sexual_violence                 32648
Physical_violence                5946
emotional_violence                651
economic_violence                 217
Harmful_Traditional_practice      188
Name: count, dtype: int64

In [36]:
sexual_violence=violence_df[violence_df['label']=='sexual_violence'].sample(n=4998, random_state=42)
violence_df=violence_df[violence_df['label']!='sexual_violence']

In [37]:
violence_df.shape

(7002, 2)

In [38]:
violence_df=pd.concat([sexual_violence,violence_df],axis=0)

In [39]:
violence_df.shape

(12000, 2)

In [40]:
hate_df['label'].value_counts()

label
1    19190
2     4163
0     1430
Name: count, dtype: int64

In [41]:
label1=hate_df[hate_df['label']==1].sample(n=6407, random_state=42)
hate_df=hate_df[hate_df['label']!=1]

In [42]:
hate_df.shape

(5593, 2)

In [43]:
hate_df=pd.concat([label1,hate_df],axis=0)

In [44]:
hate_df.shape

(12000, 2)

In [45]:
emotion_df.shape,violence_df.shape,hate_df.shape

((12000, 2), (12000, 2), (12000, 2))

In [46]:
emotion_df.reset_index(drop=True,inplace=True)
violence_df.reset_index(drop=True,inplace=True)
hate_df.reset_index(drop=True,inplace=True)

3.Label Encoding

In [47]:
label_encoder=LabelEncoder()
violence_df['label']=label_encoder.fit_transform(violence_df['label'])

In [48]:
violence_df.head()

,text,label
0,My cousin was raped by this guy Matthew. She w...,4
1,HAHAHAHAHAHAHHA I DIDN’T SEE IT THE FIRST TIME...,4
2,I remember how I almost got raped like it was ...,4
3,He raped me 👈,4
4,"A woman raped by A male: psychological horror,...",4


In [49]:
violence_df['label'].unique()

array([4, 1, 3, 0, 2])

4. Stopwords Removal

In [50]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [51]:
stop_words=set(stopwords.words('english'))

In [52]:
len(stop_words)

179

In [53]:
def remove_stopwords(text) :
    all_words=nltk.word_tokenize(text)
    filtered_words=[word for word in all_words if word.lower() not in stop_words]
    return " ".join(filtered_words)

emotion_df['text']=emotion_df['text'].apply(remove_stopwords)
violence_df['text']=violence_df['text'].apply(remove_stopwords)
hate_df['text']=hate_df['text'].apply(remove_stopwords)

5. Tokenization and Padding

In [55]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(pd.concat([emotion_df['text'],violence_df['text'],hate_df['text']]))

In [56]:
emotion_sequences=tokenizer.texts_to_sequences(emotion_df['text'])
violence_sequences=tokenizer.texts_to_sequences(violence_df['text'])
hate_sequences=tokenizer.texts_to_sequences(hate_df['text'])

In [57]:
emotion_df['text'].iloc[2]

'feel like lost mourned moved past tears relationship'

In [58]:
emotion_sequences[2:3]

[[1, 5, 321, 11854, 1207, 422, 1093, 385]]

In [60]:
max_length=50
emotion_padded=pad_sequences(emotion_sequences,maxlen=max_length,padding='post')
violence_padded=pad_sequences(violence_sequences,maxlen=max_length,padding='post')
hate_padded=pad_sequences(hate_sequences,maxlen=max_length,padding='post')

In [61]:
emotion_padded[2:3]

array([[    1,     5,   321, 11854,  1207,   422,  1093,   385,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0]], dtype=int32)

In [63]:
emotion_labels=np.array(emotion_df['label'])
violence_labels=np.array(violence_df['label'])
hate_labels=np.array(hate_df['label'])

6. Model Definition

In [64]:
emotion_input=emotion_padded
violence_input=violence_padded
hate_input=hate_padded